In [ ]:
import pandas as pd

In [ ]:
rest = pd.read_csv("resturants.csv")

In [ ]:
rest.info()

the last column location doesn have any value, then we drop

In [ ]:
rest.drop(columns=['Location'],inplace=True)
rest.head()

"Reviewers" should be integer, so we will remove () and + if they are 100+, then convert to int

In [ ]:
rest['Reviewers'] = rest['Reviewers'].str.replace(r'[()+]', '', regex=True)

In [ ]:
rest['Reviewers'] = rest['Reviewers'].astype(int)

In [ ]:
rest['Reviewers'].dtype

find foodtype if there is anything unusual

In [ ]:
rest['FoodType'].unique()

Remove all ending with 'min' (not a food type), although some food type may overlap, due to repsect to the real world type people are using, we will keep all other types

In [ ]:
rest_toremove = rest[rest['FoodType'].str.endswith('min', na=False)]['FoodType']

In [ ]:
rest = rest[~rest['FoodType'].isin(rest_toremove)]

In [ ]:
rest['FoodType'].unique()

Extract the available location info from the 'CompleteStoreName'

In [ ]:
rest['Location'] = rest['CompleteStoreName'].str.extract(r'\((.*?)\)')
rest

In [ ]:
rest.reset_index(drop=True, inplace=True)

In [ ]:
rest

Add Singapore district info 

In [ ]:
location = pd.DataFrame(rest['Location'])
location

In [ ]:
def get_district_mapping():
    """
    Returns a dictionary mapping neighborhoods to their corresponding Singapore districts
    """
    # Previous district mapping code remains the same
    return {
        # District 1 (Raffles Place, Cecil, Marina, People's Park)
        'Raffles Place': 1, 'Marina': 1, "People's Park": 1, 'Cecil': 1,
        'Raffles City': 1, 'Marina Bay': 1,
        
        # District 2 (Anson, Tanjong Pagar)
        'Anson': 2, 'Tanjong Pagar': 2,
        
        # District 3 (Queenstown, Tiong Bahru)
        'Queenstown': 3, 'Tiong Bahru': 3, 
        
        # District 4 (Telok Blangah, Harbourfront)
        'Telok Blangah': 4, 'Harbourfront': 4,
        
        # District 5 (Pasir Panjang, Clementi)
        'Pasir Panjang': 5, 'Clementi': 5,
        
        # District 6 (High Street, Beach Road)
        'High Street': 6, 'Beach Road': 6, 'North Bridge Road': 6, 
        'Bugis': 6, 'City Hall': 6, 'Arab Street': 6, 'Bras Basah': 6,
        
        # District 7 (Middle Road, Golden Mile)
        'Middle Road': 7, 'Golden Mile': 7, 'Bencoolen': 7,
        'Bugis Junction': 7, 'Bugis+': 7,
        
        # District 8 (Little India)
        'Little India': 8, 'Farrer Park': 8, 'Serangoon Road': 8,
        'Race Course': 8, 'Tekka': 8,
        
        # District 9 (Orchard, Cairnhill, River Valley)
        'Orchard': 9, 'Cairnhill': 9, 'River Valley': 9, 
        'Killiney': 9, 'Somerset': 9, 'Dhoby Ghaut': 9,
        
        # District 10 (Bukit Timah, Holland Road, Tanglin)
        'Bukit Timah': 10, 'Holland': 10, 'Tanglin': 10,
        'Holland Village': 10, 'Sixth Avenue': 10, '6th Avenue': 10,
        
        # District 11 (Novena, Thomson, Watten Estate)
        'Novena': 11, 'Thomson': 11, 'Watten Estate': 11,
        'Newton': 11, 'Upper Thomson': 11,
        
        # District 12 (Balestier, Toa Payoh, Serangoon)
        'Balestier': 12, 'Toa Payoh': 12, 'Serangoon': 12,
        
        # District 13 (Macpherson, Braddell)
        'Macpherson': 13, 'Braddell': 13,
        
        # District 14 (Geylang, Paya Lebar, Sims)
        'Geylang': 14, 'Paya Lebar': 14, 'Sims': 14,
        
        # District 15 (Katong, Joo Chiat, Amber Road)
        'Katong': 15, 'Joo Chiat': 15, 'Amber Road': 15,
        
        # District 16 (Bedok, Upper East Coast)
        'Bedok': 16, 'Upper East Coast': 16,
        
        # District 17 (Loyang, Changi)
        'Loyang': 17, 'Changi': 17,
        
        # District 18 (Tampines, Pasir Ris)
        'Tampines': 18, 'Pasir Ris': 18,
        
        # District 19 (Serangoon Garden, Hougang, Punggol)
        'Serangoon Garden': 19, 'Hougang': 19, 'Punggol': 19,
        
        # District 20 (Bishan, Ang Mo Kio)
        'Bishan': 20, 'Ang Mo Kio': 20, 'AMK': 20,
        
        # District 21 (Upper Bukit Timah, Clementi Park)
        'Upper Bukit Timah': 21, 'Clementi Park': 21,
        
        # District 22 (Jurong)
        'Jurong': 22, 'Jurong East': 22, 'Jurong West': 22,
        
        # District 23 (Hillview, Bukit Panjang)
        'Hillview': 23, 'Bukit Panjang': 23, 'Dairy Farm': 23,
        
        # District 24 (Lim Chu Kang, Tengah)
        'Lim Chu Kang': 24, 'Tengah': 24,
        
        # District 25 (Kranji, Woodlands)
        'Kranji': 25, 'Woodlands': 25,
        
        # District 26 (Upper Thomson, Springleaf)
        'Springleaf': 26,
        
        # District 27 (Yishun, Sembawang)
        'Yishun': 27, 'Sembawang': 27,
        
        # District 28 (Seletar)
        'Seletar': 28
    }
'''
def get_district_coordinates():
    """
    Returns a dictionary of district coordinates (approximate center points)
    """
    return {
        1: {'lat': 1.2830, 'lng': 103.8513},  # Raffles Place
        2: {'lat': 1.2743, 'lng': 103.8459},  # Tanjong Pagar
        3: {'lat': 1.2847, 'lng': 103.8320},  # Tiong Bahru
        4: {'lat': 1.2708, 'lng': 103.8354},  # Harbourfront
        5: {'lat': 1.2880, 'lng': 103.7862},  # Pasir Panjang
        6: {'lat': 1.2966, 'lng': 103.8565},  # Beach Road
        7: {'lat': 1.3011, 'lng': 103.8543},  # Middle Road
        8: {'lat': 1.3074, 'lng': 103.8517},  # Little India
        9: {'lat': 1.3048, 'lng': 103.8318},  # Orchard
        10: {'lat': 1.3158, 'lng': 103.8160}, # Bukit Timah
        11: {'lat': 1.3204, 'lng': 103.8439}, # Novena
        12: {'lat': 1.3340, 'lng': 103.8472}, # Toa Payoh
        13: {'lat': 1.3280, 'lng': 103.8773}, # Macpherson
        14: {'lat': 1.3168, 'lng': 103.8930}, # Geylang
        15: {'lat': 1.3040, 'lng': 103.9050}, # Katong
        16: {'lat': 1.3240, 'lng': 103.9279}, # Bedok
        17: {'lat': 1.3644, 'lng': 103.9915}, # Changi
        18: {'lat': 1.3546, 'lng': 103.9450}, # Tampines
        19: {'lat': 1.3573, 'lng': 103.8763}, # Serangoon Garden
        20: {'lat': 1.3695, 'lng': 103.8483}, # Ang Mo Kio
        21: {'lat': 1.3397, 'lng': 103.7747}, # Upper Bukit Timah
        22: {'lat': 1.3329, 'lng': 103.7436}, # Jurong
        23: {'lat': 1.3690, 'lng': 103.7644}, # Bukit Panjang
        24: {'lat': 1.4206, 'lng': 103.7179}, # Lim Chu Kang
        25: {'lat': 1.4382, 'lng': 103.7891}, # Woodlands
        26: {'lat': 1.3998, 'lng': 103.8261}, # Upper Thomson
        27: {'lat': 1.4504, 'lng': 103.8200}, # Yishun
        28: {'lat': 1.4171, 'lng': 103.8659}  # Seletar
    }
'''
def get_district(location, district_mapping):
    """
    Returns the district number based on location name
    """
    if pd.isna(location):
        return None
        
    location = location.strip()
    
    # Direct lookup
    for area, district in district_mapping.items():
        if area.lower() in location.lower():
            return district
            
    # Special cases
    if any(x in location.lower() for x in ['amk', 'ang mo']):
        return 20  # Ang Mo Kio
    if 'thomson' in location.lower():
        return 11  # Thomson area
    
    return None

def process_locations(df):
    """
    Process the locations DataFrame and add district numbers and coordinates
    """
    # Create mappings
    district_mapping = get_district_mapping()
    district_coordinates = get_district_coordinates()
    
    # Add district column
    df['District'] = df['Location'].apply(lambda x: get_district(x, district_mapping))
    
    # Add coordinate columns
    #df['Latitude'] = df['District'].apply(lambda x: district_coordinates[x]['lat'] if pd.notnull(x) else None)
    #df['Longitude'] = df['District'].apply(lambda x: district_coordinates[x]['lng'] if pd.notnull(x) else None)
    
    return df

# Read and process the data
df = location
df = process_locations(df)


In [ ]:
df.reset_index(drop=True, inplace=True)

To prepare the data for exact latitude and longitude

In [ ]:
df_to_process = rest['Location'] + ' Singapore'
df_to_process = pd.DataFrame(df_to_process)
df_to_process

Slice the data into trunks for smooth process

In [ ]:
num_rows = df_to_process.shape[0]
df_collection = []
for i in range(0,num_rows,50):
    df_collection.append(df_to_process[i:i+50])
df_geo = []

Get extact geo data

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# Function to fetch geolocation
def get_geocode(location, retries=2):
    for attempt in range(retries):
        try:
            geo = geolocator.geocode(location)
            if geo:
                print(f"location got with: {geo}")
                return geo.latitude, geo.longitude
        except GeocoderTimedOut:
            time.sleep(2)  # Wait before retrying
    print(f"{location} not got")
    return None, None

# Apply the function to the location column
for location in df_collection:
    
    location[['Latitude', 'Longitude']] = location['Location'].apply(lambda x: pd.Series(get_geocode(x)))
    df_geo.append(location)

In [ ]:
df_geo

Add district and geo info into the main df

In [ ]:
rest[['District']] = df[['District']]

In [ ]:
rest[['Location', 'Latitude','Longitude']] = df_geo[['Location', 'Latitude','Longitude']]

In [ ]:
rest

In [ ]:
rest.to_csv("rest_cleaned_geo.csv")